### Downloading CRS reports and metadata

Source: https://www.everycrsreport.com

In [1]:
#!/usr/bin/env python3

# Sample script to download CRS reports from EveryCRSReport.com.
#
# EveryCRSReport publishes a listing file at
# https://www.everycrsreport.com/reports.csv which has the number,
# last publication date, relative URL to a report metadata JSON
# file, and the SHA1 hash of the metadata file.
#
# We use that file to download new reports into:
#
# reports/reports/xxxxxx.json
# reports/files/yyyyy.pdf
# reports/files/yyyyy.html
#
# This script was written in Python 3.

import hashlib
import urllib.request
import io
import csv
import os, os.path
import json
import time

api_base_url = "https://www.everycrsreport.com/"

def download_file(url, fn, expected_digest):
    # Do we have it already?
    if os.path.exists(fn):
        # Compute the SHA1 hash of the existing file's contents,
        # if we are given a hash.
        with open(fn, 'rb') as f:
            hasher = hashlib.sha1()
            hasher.update(f.read())
            digest = hasher.hexdigest()

        # Is the existing file up to date?
        if digest == expected_digest or expected_digest is None:
            # No need to download
            return

    # Download and save the file.
    print(fn + "...")
    try:
        with urllib.request.urlopen(url) as resp:
            data = resp.read()
            with open(fn, 'wb') as f:
                f.write(data)
    except urllib.error.HTTPError as e:
        print("", e)
    time.sleep(1)

# Ensure output directories exist.
os.makedirs("reports/reports", exist_ok=True)
os.makedirs("reports/files", exist_ok=True)

In [3]:
# Execute an HTTP request to get the CSV listing file.
with urllib.request.urlopen(api_base_url + "reports.csv") as resp:
    # Parse it as a CSV file.
    reader = csv.DictReader(io.StringIO(resp.read().decode("utf8")))

In [4]:
# Fetch reports.
for report in reader:
    # Where will we save this report?
    metadata_fn = "reports/" + report["url"] # i.e. reports/reports/R1234.json

    # Download it if we don't have it or it's modified.
    download_file(api_base_url + report["url"], metadata_fn, report["sha1"])

    # Also download the PDF/HTML files for the report.
    #
    # While we could get the most recent PDF and HTML file names from the
    # CSV file directly (report["latestPDF"], report["latestHTML"]), this
    # script demonstrates how to get all past versions of a report also.
    #
    # Parse the metadata JSON file to figure out what the PDF/HTML file names are.
    
    with open(metadata_fn) as f:
        metadata = json.load(f)

        # Each report may have multiple versions published.
        for version in metadata["versions"]:
            # Each report version is published in zero or more file formats.
            for report_file in version["formats"]:
                print(report_file['format'])
                if report_file['format'] == "HTML":
                    # Where will we save this file?
                    file_fn = "reports/" + report_file["filename"]

                    # Download it if we don't have it or it's modified.
                    download_file(api_base_url + report_file["filename"], file_fn, None)

reports/reports/IN10931.json...
HTML
reports/files/20180718_IN10931_4c47451b271423b7aab8e0d46fe6373d4ec60bdd.html...
PDF
reports/reports/R45260.json...
HTML
reports/files/20180718_R45260_cf336e2767414cad334f53b1eb547c23d6f33958.html...
PDF
reports/reports/R45261.json...
PDF
HTML
reports/files/20180718_R45261_1551ff045835abb690731f99eec6a785e7d72449.html...
reports/reports/R45259.json...
HTML
reports/files/20180718_R45259_482e024fa92fc28a5e1a2674b622d87d9bdc54b8.html...
PDF
reports/reports/R44017.json...
HTML
reports/files/20180718_R44017_04eb101d6bc152ba8d05aab7b1f44aa59b82ff3f.html...
PDF
HTML
reports/files/20180523_R44017_2754fb761f707174dd260221092acbb890603a6a.html...
PDF
HTML
reports/files/20180320_R44017_3528d62794f7c476f1d36c6c57b4b8f9166aec59.html...
PDF
HTML
reports/files/20180119_R44017_b00c3cb2b0a764a3590ee17cf3aa94484791ea76.html...
PDF
HTML
reports/files/20171107_R44017_23e8cb77c673830e8e90188a0dbfbea133731644.html...
PDF
HTML
reports/files/20170922_R44017_6b1402674aace01f

HTML
reports/files/20170420_R43310_d6b054fab777b0adefb733c810a09139890e6fad.html...
HTML
reports/files/20161102_R43310_c23472efbb9d9d3d7bfaf0e7798ea52ffee5e41a.html...
HTML
reports/files/20160527_R43310_c79b55144b5313d8f7dc3eb7a1efc6daee1f092c.html...
HTML
reports/files/20151230_R43310_66736fbafec1aab3a68686ad27fd6195f05eb530.html...
PDF
HTML
reports/files/20150908_R43310_7769650fd9e5bfe543b84cb315887b856af6dfb3.html...
PDF
HTML
reports/files/20150205_R43310_25b34a42bc254358a263e21f250395fa090b42e2.html...
reports/reports/RL31457.json...
HTML
reports/files/20180713_RL31457_99463929b7922a8c0faa47c5ac1bc9172987739a.html...
PDF
HTML
reports/files/20180309_RL31457_08ddec44ab7ae685cc5507938a0cf2bb0a1e980a.html...
PDF
HTML
reports/files/20160330_RL31457_194d3dd7d67ac227a15366f0bc547626c6056127.html...
PDF
HTML
reports/files/20100910_RL31457_4416d18d9452e95150949fe3d90fe53f9c45fd21.html...
PDF
PDF
HTML
reports/files/20080327_RL31457_96576f003ea8d7b6fd36da2a8e990ee56a4745b6.html...
PDF
HTML
re

PDF
HTML
reports/files/20110316_RS22478_ea71212bf8f93f66c9652b0607d5afe8b5b418fe.html...
PDF
HTML
reports/files/20110113_RS22478_85c4032476678a9edfc5fa6ac50703cdd25c363f.html...
PDF
HTML
reports/files/20101223_RS22478_8bc55f7fe4e59c6cac38a75bbbda8dbc3e2f995b.html...
PDF
HTML
reports/files/20100721_RS22478_9aa229c9294da95dbac89edf3280c01e81b8477b.html...
PDF
HTML
reports/files/20100702_RS22478_cbcab526673f1f4201016c554e8f9a2adc71b2bc.html...
PDF
HTML
reports/files/20100518_RS22478_f9258574a84a2bd35db7cd83d2132c1d07f17bfe.html...
PDF
HTML
reports/files/20100426_RS22478_1a61cc04ac5710fc8c09dc17043123c059557f29.html...
PDF
HTML
reports/files/20100413_RS22478_22299e231075fa940a3ec9d5fbf0d4fd8d7d75ee.html...
PDF
HTML
reports/files/20091223_RS22478_00146ffa9663644928887531c95439ac692fd4f4.html...
PDF
HTML
reports/files/20091117_RS22478_218830291aa66b05de1de3b326c63f0567fb45d0.html...
PDF
HTML
reports/files/20091020_RS22478_6c5c35fab8996741c343ef3ff380ff6a09dd1686.html...
PDF
HTML
reports/file

PDF
HTML
reports/files/20161013_RL34391_893c524c84a19a73a6afe0447588c3553ed747d5.html...
PDF
HTML
reports/files/20160708_RL34391_a7928e75b6ba73a7806b85c8347da09267595a23.html...
PDF
HTML
reports/files/20160527_RL34391_7534358da89326173ab613d3ccc9189749f0acbc.html...
PDF
HTML
reports/files/20160415_RL34391_6471a316f8cf03a5a6ac6126ad3f64181523fdfd.html...
PDF
HTML
reports/files/20160404_RL34391_fa567ba2e49b4d68a1e59c36e2a3f15dc36bd756.html...
PDF
HTML
reports/files/20160321_RL34391_f2293a3e938934769df37bf3bae41cae17ac328e.html...
PDF
PDF
HTML
reports/files/20160115_RL34391_e1e1f5087ade05464e0d59a1408cdc89e4ed839a.html...
PDF
HTML
reports/files/20151120_RL34391_655e2ac7e075cccf43067d40f041c5962071ff8d.html...
PDF
HTML
reports/files/20150925_RL34391_d4b0f82e79c74bb2dc1b5e35cc4f13103e23ea88.html...
PDF
HTML
reports/files/20150902_RL34391_cd1be216980eba3cbcb589c1ea1b7f89ed9e4ace.html...
PDF
HTML
reports/files/20150728_RL34391_a0feb88503a6ea1e4dc2c6c2a79fedf437f13c8c.html...
PDF
HTML
reports/

PDF
HTML
reports/files/20091209_RS21513_3ef1cccc18d97711ba1be5ddf2654f4e233d4a79.html...
PDF
HTML
reports/files/20090520_RS21513_94dc39c2eb0f46235b92ccf0bbd0cb3e02e16aa2.html...
PDF
HTML
reports/files/20090403_RS21513_fd7a87557e3c6f50d81e5714e76a2de6f073fb0c.html...
PDF
HTML
reports/files/20081206_RS21513_b9c0ee0370f3eaadb3368dd4c29bc66b2e3f6885.html...
PDF
HTML
reports/files/20080708_RS21513_7ae3e8f5bf548a74f723526dc17a9d5ff507bf2e.html...
PDF
HTML
reports/files/20071102_RS21513_649c30a88abdbb2868ed87a4b040a43dc08d4b1a.html...
PDF
HTML
reports/files/20060705_RS21513_6484e0ab6139600ff66c24cb105507e7a7c4aac0.html...
PDF
HTML
reports/files/20060524_RS21513_0c3386173017ae39e0e397e9bd7f4a84afe9c517.html...
PDF
HTML
reports/files/20050629_RS21513_57dfa9ee003efd2587e9a61274eaf3b09f93b79f.html...
PDF
HTML
reports/files/20050518_RS21513_0fe049d75f34e726749a1f7d398b2be17eb90d78.html...
reports/reports/R41153.json...
HTML
reports/files/20180709_R41153_c3a22fc7f1178377867ff82e255cb746e4d3426f.htm

...

PDF
PDF
HTML
reports/files/20080312_RL34266_60ce23131dc070bb20c0e289bd5f87d4100cae17.html...
PDF
HTML
reports/files/20080107_RL34266_52c29918987d37f807a3d4f7d18b7e4e83f6bcef.html...
PDF
HTML
reports/files/20071129_RL34266_f423f65b6f785bde699272f487226e0b2dedb7b6.html...
reports/reports/R40160.json...
HTML
reports/files/20090223_R40160_a42d5814304f036da891dd503a4b215501a84c19.html...
PDF
reports/reports/R40235.json...
HTML
reports/files/20090223_R40235_402d02566a4b00bff49a7fad15e3e8cc90482049.html...
PDF
reports/reports/RL33876.json...
HTML
reports/files/20090220_RL33876_ca424006a3d72e20489eb22809b6bba4b56da625.html...
PDF
PDF
HTML
reports/files/20081010_RL33876_4d5a1d28486887a6e0fa881d3e21cb0aa26a7405.html...
PDF
HTML
reports/files/20070214_RL33876_17fc47ce1051166a9c85f645dfc0875eef0e7658.html...
reports/reports/RS22653.json...
PDF
HTML
reports/files/20090219_RS22653_83ba24856b61d6bf793effb3ff06baee9ca6d6c1.html...
PDF
HTML
reports/files/20080714_RS22653_4858cfa991823acbc2cbc06465b652d

PDF
HTML
reports/files/20080417_RL31988_9cc14d53a526a318f0791410814b995587be087f.html...
PDF
HTML
reports/files/20071210_RL31988_c869e269342a97bb73c36f16687703259f4607e8.html...
PDF
HTML
reports/files/20070316_RL31988_6c14fbba114a34450f9b8c44243b2fa758e246ba.html...
PDF
HTML
reports/files/20070214_RL31988_fe2d291e0b19c8de3d3286f3f5754be58e2f543e.html...
PDF
HTML
reports/files/20060407_RL31988_206ea3339ee6811047d4e75a5b6306d193b0ca95.html...
PDF
HTML
reports/files/20050203_RL31988_d027577c67e3dbb02c7b0df705b437d2110cd2d4.html...
PDF
HTML
reports/files/20031001_RL31988_18445999657538273aba4fc652f7b48fedc04ef7.html...
PDF
HTML
reports/files/20030708_RL31988_adcd3e78b1064a0bf3ae5b38bc9dd3c85326b205.html...
reports/reports/RL34749.json...
PDF
HTML
reports/files/20090209_RL34749_0ecda90c9ac9a8e7d6590c48cf237da67b0eb182.html...
PDF
HTML
reports/files/20081118_RL34749_e5fd38762e26ba26899d9f796a1b0b02b70da59a.html...
reports/reports/R40225.json...
HTML
reports/files/20090209_R40225_8cac7f792942

PDF
PDF
HTML
reports/files/20080825_RL34569_b560bbf204de3bd83c74cece8d50d3bb01a143d5.html...
PDF
HTML
reports/files/20080714_RL34569_2886120cd88447c157c15b9e133194a69e310948.html...
reports/reports/RL34556.json...
HTML
reports/files/20090129_RL34556_5e59b1115650a54f0d7cddecd31530dad08460ec.html...
PDF
PDF
HTML
reports/files/20080627_RL34556_5defe24df0b435ac109a1e68f0a7ca231de90399.html...
reports/reports/R40190.json...
HTML
reports/files/20090129_R40190_5fa83bf393f93f176ac1d6eba46c7504677ea457.html...
PDF
reports/reports/RL33683.json...
HTML
reports/files/20090129_RL33683_72c6c2555b0644cad6ea1a73067312773ebaf209.html...
PDF
PDF
HTML
reports/files/20070312_RL33683_9b029913ef5948d0e4784a663cc1b2390321b171.html...
PDF
HTML
reports/files/20061113_RL33683_0e9f7f762b97b8e7b24884e45b417c95a4168462.html...
reports/reports/RL34123.json...
HTML
reports/files/20090129_RL34123_8d19e957d658f9334692bd85b704f84a2dbd94ba.html...
PDF
PDF
HTML
reports/files/20081024_RL34123_efcb25ce168e5ff3c57f92d159e07

PDF
PDF
HTML
reports/files/20081024_RS22928_daa138714e26310376b610f81870328aaab2db8b.html...
PDF
HTML
reports/files/20080806_RS22928_d909946f0bc5df2873d84926582f4fae55077fce.html...
reports/reports/RS22895.json...
HTML
reports/files/20090126_RS22895_0ce87a4327fda7e27827fd9f1bdaa6e8ead5ac9c.html...
PDF
PDF
HTML
reports/files/20080612_RS22895_09f8d6abc0e10e10c0b565df6aeb4e4528a73261.html...
reports/reports/RL33828.json...
HTML
reports/files/20090126_RL33828_501f3db0a18d7e8e1b5ece36dbca528cbc305262.html...
PDF
PDF
HTML
reports/files/20080702_RL33828_12c1ca7059d91c9b492f4a0bef4ab11f6d17ed68.html...
PDF
HTML
reports/files/20070831_RL33828_1b0a695b1c452635373075655fae149d7c742fe0.html...
PDF
HTML
reports/files/20070622_RL33828_c4ceae827307672452f22d6cb852aee0a56c766d.html...
PDF
HTML
reports/files/20070123_RL33828_cfcd98f719a0fb660872185113e64279a5fc497c.html...
reports/reports/RL32657.json...
HTML
reports/files/20090123_RL32657_d5dc7909b9103b4d1d88b0aaeb49840a54eb0d5a.html...
PDF
PDF
HTML
r

KeyboardInterrupt: 

### Parsing reports

20k is sufficient! The reports are used as a comparison for the type of language used in speeches and 20k is more than sufficient for that purpose.

In [15]:
import codecs
from bs4 import BeautifulSoup
import re

def parse_text(text):
    f=codecs.open(text, 'rb', encoding = 'utf-8', errors = 'ignore')
    document= BeautifulSoup(f.read(), 'lxml').get_text()
    document_2 = document.replace("\n", " ").replace("\n", " ")
    #print(document)
    return document_2

def check_copyright(soup_text):
    copyright_list = re.findall('©', soup_text)
    if copyright_list:
        print('Copyright warning!')
        return True
    else:
        return False

In [18]:
import pickle

path = '/home/ubuntu/Notebooks/reports/files'

list_documents = []
for filename in os.listdir(path):
    document = parse_text('reports/files/' + str(filename))
    if check_copyright(document) is False:
        list_documents.append(document)
    else:
        print(filename)
    if len(list_documents) % 1000 == 0:
        print('Number of documents: ', len(list_documents))
    if len(list_documents) == 20000:
        break


Copyright warning!
20090402_R40488_d171cca48a28ec803750ae8e5e7c78d9383b45eb.html
Copyright warning!
20090206_RL33704_552940c6c8531d888e7f259b04f390ff19da76b8.html
Number of documents:  1000
Copyright warning!
20101229_RL33577_2f8a0dcb0abf14e5e1fde20b3b56afad32c2ead5.html
Copyright warning!
20110621_RL32571_62bbb44c359b63d78dc35e65e85713fc8e23705b.html
Number of documents:  2000
Copyright warning!
20110906_RL33577_deb66dd38157158016e0117a69898d433212c620.html
Number of documents:  3000
Copyright warning!
20150519_R44039_5608e9bb83fddfc29d9c56e7f942219d25506f7d.html
Copyright warning!
20141224_R42784_32a8e5957efbc12e2ba117282849a8d30d59ddbb.html
Number of documents:  4000
Copyright warning!
20140127_RS20840_12da6351aa30bd5710a1c5d768f5c1ef5d3a3026.html
Number of documents:  5000
Copyright warning!
20100325_RL33902_588836cec4bec85f1d09201e45b5b056074b7c3c.html
Number of documents:  6000
Copyright warning!
20090116_R40167_e9870542b13b17a56fc55f37a9f39ea483ea85b0.html
Copyright warning!
201

In [19]:
output = open('crs_reports.pkl', 'wb')
pickle.dump(list_documents, output)

output.close()